**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [199]:
!pip install keras-flops

In [201]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [202]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [203]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [204]:
# Преобразуйте метки классов в one_hot формат
from sklearn.preprocessing import OneHotEncoder

onehotencoder = OneHotEncoder()

y_train = (y_train == np.arange(CLASSES)).astype(np.float32) 
y_val = (y_val == np.arange(CLASSES)).astype(np.float32) 

In [205]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [225]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32, 32, 3]),
    
    tf.keras.layers.Conv2D(8, kernel_size=(5, 5), strides=(2, 2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Conv2D(4, kernel_size=(3, 3), strides=(2, 2), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [226]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")


=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
flops: Number of float operations. Note: Please read the implementation for the math behind it.

Profi

2023-01-25 14:30:45.773787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 14:30:45.774173: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-25 14:30:45.774288: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2023-01-25 14:30:45.774832: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 14:30:45.775259: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-25 14:30:45.775599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] su

lization_73/batchnorm/Rsqrt (256/256 flops)
  sequential_50/batch_normalization_72/batchnorm/sub (256/256 flops)
  sequential_50/batch_normalization_72/batchnorm/mul_2 (256/256 flops)
  sequential_50/batch_normalization_72/batchnorm/mul (256/256 flops)
  sequential_50/batch_normalization_73/batchnorm/mul_2 (128/128 flops)
  sequential_50/batch_normalization_73/batchnorm/sub (128/128 flops)
  sequential_50/batch_normalization_73/batchnorm/mul_1 (128/128 flops)
  sequential_50/batch_normalization_73/batchnorm/mul (128/128 flops)
  sequential_50/batch_normalization_73/batchnorm/add_1 (128/128 flops)
  sequential_50/batch_normalization_73/batchnorm/add (128/128 flops)
  sequential_50/dense_57/BiasAdd (128/128 flops)
  sequential_50/dense_58/BiasAdd (100/100 flops)

======================End of Report==========================


In [227]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_83 (Conv2D)           (None, 16, 16, 8)         608       
_________________________________________________________________
batch_normalization_70 (Batc (None, 16, 16, 8)         32        
_________________________________________________________________
activation_119 (Activation)  (None, 16, 16, 8)         0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 8, 8, 4)           292       
_________________________________________________________________
batch_normalization_71 (Batc (None, 8, 8, 4)           16        
_________________________________________________________________
activation_120 (Activation)  (None, 8, 8, 4)           0         
_________________________________________________________________
flatten_37 (Flatten)         (None, 256)             

In [228]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [229]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=16
)

Epoch 1/16
391/391 [==============================] - 3s 6ms/step - loss: 3.6022 - accuracy: 0.1534 - val_loss: 3.7246 - val_accuracy: 0.1592
Epoch 2/16
391/391 [==============================] - 2s 5ms/step - loss: 3.1143 - accuracy: 0.2375 - val_loss: 3.3225 - val_accuracy: 0.2125
Epoch 3/16
391/391 [==============================] - 2s 5ms/step - loss: 2.8720 - accuracy: 0.2831 - val_loss: 3.1471 - val_accuracy: 0.2446
Epoch 4/16
391/391 [==============================] - 2s 5ms/step - loss: 2.6771 - accuracy: 0.3233 - val_loss: 3.1257 - val_accuracy: 0.2574
Epoch 5/16
391/391 [==============================] - 2s 6ms/step - loss: 2.5089 - accuracy: 0.3567 - val_loss: 2.9975 - val_accuracy: 0.2888
Epoch 6/16
391/391 [==============================] - 2s 5ms/step - loss: 2.3601 - accuracy: 0.3904 - val_loss: 2.9675 - val_accuracy: 0.2956
Epoch 7/16
391/391 [==============================] - 2s 5ms/step - loss: 2.2315 - accuracy: 0.4151 - val_loss: 3.0004 - val_accuracy: 0.2966
Epoch 